# Self-calibration

We have partially automated the cleaning and self-calibration process.  This document describes how you can use the self-cal scripts to clean a target field.

The general process is:

1. Produce a dirty image
1. Create a mask based on that dirty image, using either the [region-based threshold approach](https://github.com/ALMA-IMF/reduction/blob/master/reduction/make_custom_mask.py) (see below) or CRTF region files (i.e., regions drawn onto the image as you normally do in an interactive clean session).
1. Clean within that first mask region to a moderate depth
1. Create a new mask
1. Perform a phase-only self-calibration
1. Reject any _fields_ in the self-calibration with inadequate signal-to-noise using [heuristics](https://github.com/ALMA-IMF/reduction/blob/master/reduction/selfcal_heuristics.py).  This stage is essential for our mosaiced fields, in which some pointings will have no signal at all.
1. Apply the self-calibration solutions
1. Clean with the new mask to a somewhat greater depth
1. Create a new mask
1. Repeat steps 5-9 above

Then, after a few iterations of phase-only self-calibration, perform one or more iterations of amplitude self-calibration

## Imaging Parameters

To make the above approach straightforward, this is all wrapped into a [script](https://github.com/ALMA-IMF/reduction/blob/master/reduction/continuum_imaging_selfcal.py), and users only need modify the [parameters](https://github.com/ALMA-IMF/reduction/blob/master/reduction/imaging_parameters.py).

The parameters include the following global defaults as inputs to `tclean`:

```
                      {'threshold': '1mJy',
                       'pblimit': 0.1,
                       'niter': 10000,
                       'robust': robust,
                       'weighting': 'briggs',
                       'scales': [0,3,9],
                       'gridder': 'mosaic',
                       'specmode': 'mfs',
                       'deconvolver': 'mtmfs',
                       'nterms': 2,
                      }
```                      
These can be overridden in the `imaging_parameters.py` file, and other `tclean` parameters can be added.
The imaging parameters are specified as a dictionary, so each field will have a dictionary like the one above.  The keys should have the format `{field}_{band}_{array}_robust{robust}`, where `{field}` is the field name (e.g., `W51-E`, `G333.60`, etc), `{band}` is the band name (`B3` or `B6`), `{array}` is the array name (`7M12M` or `12M`), and `{robust}` is the robustness value (most likely 0, -2, or 2).  

If you would like an imaging parameter to change with each iteration, you can specify that using a dictionary where the key is the iteration number.  For example, if you want to clean to 1 mJy in the first iteration (before any self-calibration has been applied), then to 0.5 mJy after the first self-cal iteration, you would specify it as such:
```
imaging_parameters_nondefault = {'W51IRS2_B3_12M_robust0': {'niter': {0: '1mJy', 1: '0.5mJy'} } }
```
Note that the `imaging_parameters_nondefault` variable name is used within `imaging_parameters.py` already; you will just add to it (don't overwrite what's already there).

## Self-calibration parameters
The self-calibration parameters are specified in a similar way.  The default selfcal parameters are:
```
                            {'solint': 'int',
                             'gaintype': 'G',
                             'solnorm': True,
                             'calmode': 'p'}
```
The default setup is to do 4 iterations of phase-only self-calibration with the above parameters

To override these, for example, to add the amplitude self-calibration steps, you add new entries to the `selfcal_parameters` dictionary, e.g.:
```
selfcal_pars['W51-E_B6_12M_robust0'][5] = {'solint': 'inf',
                                           'gaintype': 'G',
                                           'calmode': 'ap',
                                          }
```
The first key is the same format as for imaging parameters (`{field}_{band}_{array}_robust{robust}`).  The second key, in this example the number `5`, is the self-calibration iteration number.

## Running the code

### Setup
To run the code, you should copy the ALMAIMF reduction directory somewhere onto the machine where you'll be running it.  This is best achieved by the command 
```
git clone https://github.com/ALMA-IMF/reduction.git
```
or
```
git clone git@github.com:ALMA-IMF/reduction.git
```
depending on your setup.

You then need to specify two important environmental variables:
 * `EXCLUDE_7M=True` will image only the 12m data.  It must be capitalized, and the value must be "True"; if the variable is unset or if it has any value other than "True", the 7m and 12m data will be jointly imaged 
 * `ALMAIMF_ROOTDIR` is the full path to the directory containing `continuum_imaging_selfcal.py`, i.e., the `reduction/` subdirectory of the ALMA-IMF reduction package
 
You have to have the measurement sets created before you can run this code; see [the imaging instructions](https://github.com/ALMA-IMF/reduction/blob/master/reduction/Imaging_instructions.txt).

### Execution
To execute the code, simply run the file [continuum_imaging_selfcal.py](https://github.com/ALMA-IMF/reduction/blob/master/reduction/continuum_imaging_selfcal.py) with `execfile` or `%run -i`.

If you execute the code, then discover that there are some parameters you want to change or modify, there are several files you must remove:
 * for each imaging iteration you wish to redo, remove all files with the appropriate suffix in `imaging_results/`.  e.g., if you want to re-do the first self-calibration iteration, `rm -r imaging_results/{fieldname}_*_selfcal1.*
 * for each self-calibration iteration to redo, remove the corresponding `.cal` file in the working directory.  The calibration tables are named using the following scheme:
 `'{basename}_{arrayname}_{caltype}{selfcaliter}_{solint}.cal'`, where `{basename}` is the first part of the measurement set filename, arrayname is `7M12M` or `12M`, `caltype` is `amp` or `phase`, `selfcaliter` is the iteration number, and `solint` is the solution interval.
 
The script is capable of picking up where you left off: if you want to re-do iteration 5, but you are satisfied with iterations 0-4, just remove the files for iteration 5.  The script will ignore the preceding files.